In [1]:
# Define the AWS env variables if you are using AWS Auth:
%env AWS_REGION= region
%env AWS_ACCESS_KEY_ID= key
%env AWS_SECRET_ACCESS_KEY= secret

env: AWS_REGION=region
env: AWS_ACCESS_KEY_ID=key
env: AWS_SECRET_ACCESS_KEY=secret


In [4]:
import pyspark
from pyspark.sql import SparkSession
import os


conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
    
    # first we will define the packages that we need. Iceberg Spark runtime
        .set('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.2.0,software.amazon.awssdk:bundle:2.17.178,software.amazon.awssdk:url-connection-client:2.17.178')
        
    # This property allows us to add any extensions that we want to use
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')
    
    # configures a new catalog to a particular implementation of SparkCatalog
        .set('spark.sql.catalog.glue', 'org.apache.iceberg.spark.SparkCatalog')
    
    # particular type of catalog we are using
        .set('spark.sql.catalog.glue.catalog-impl', 'org.apache.iceberg.aws.glue.GlueCatalog')
    
    # engine writes to the warehouse
        .set('spark.sql.catalog.glue.warehouse', 's3://my-bucket/warehouse/')
    
    # changes IO impl of catalog, mainly for changing writing data to object storage
        .set('spark.sql.catalog.glue.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
)

## Start Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

:: loading settings :: url = jar:file:/home/docker/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/docker/.ivy2/cache
The jars for the packages stored in: /home/docker/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.3_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0f4b8979-2efd-4aa0-8ad7-f14ea659cb5e;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.2.0 in central
	found software.amazon.awssdk#bundle;2.17.178 in central
	found software.amazon.eventstream#eventstream;1.0.1 in central
	found software.amazon.awssdk#url-connection-client;2.17.178 in central
	found software.amazon.awssdk#utils;2.17.178 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found software.amazon.awssdk#annotations;2.17.178 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found software.amazon.awssdk#http-client-spi;2.17.178 in central
	found software

23/10/11 15:00:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/10/11 15:00:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
Spark Running


# CREATE Iceberg table 'churn'

In [8]:
spark.sql(
    """CREATE TABLE IF NOT EXISTS glue.test.churn
            (State STRING, Account_length STRING, Area_code STRING, International_plan STRING, Voicemail_plan STRING, Number_vmail_messages STRING, Total_day_minutes STRING, Total_day_calls STRING, Total_day_charge STRING, Total_eve_minutes STRING, Total_eve_calls STRING, Total_eve_charge STRING, Total_night_minutes STRING, Total_night_calls STRING, Total_night_charge STRING, Total_intl_minutes STRING, Total_intl_calls STRING, Total_intl_charge STRING, Customer_service_calls STRING, Churn STRING) USING iceberg"""
)

DataFrame[]

In [9]:
spark.sql(
    """CREATE OR REPLACE TEMPORARY VIEW churnview USING csv
            OPTIONS (path "churndata.csv", header true)"""
)
spark.sql("INSERT INTO glue.test.churn SELECT * FROM churnview")

DataFrame[]

In [10]:
spark.sql("SELECT * FROM glue.test.churn").toPandas()

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


,State,Account_length,Area_code,International_plan,Voicemail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn
0,LA,117,408,No,No,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,FALSE
1,IN,65,415,No,No,0,129.1,137,21.95,228.5,83,19.42,208.8,111,9.4,12.7,6,3.43,4,TRUE
2,NY,161,415,No,No,0,332.9,67,56.59,317.8,97,27.01,160.6,128,7.23,5.4,9,1.46,4,TRUE
3,SC,111,415,No,No,0,110.4,103,18.77,137.3,102,11.67,189.6,105,8.53,7.7,6,2.08,2,FALSE
4,HI,49,510,No,No,0,119.3,117,20.28,215.1,109,18.28,178.7,90,8.04,11.1,1,3.0,1,FALSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,SC,79,415,No,No,0,134.7,98,22.9,189.7,68,16.12,221.4,128,9.96,11.8,5,3.19,2,FALSE
931,AZ,192,415,No,Yes,36,156.2,77,26.55,215.5,126,18.32,279.1,83,12.56,9.9,6,2.67,2,FALSE
932,WV,68,415,No,No,0,231.1,57,39.29,153.4,55,13.04,191.3,123,8.61,9.6,4,2.59,3,FALSE
933,RI,28,510,No,No,0,180.8,109,30.74,288.8,58,24.55,191.9,91,8.64,14.1,6,3.81,2,FALSE


# 1. Reliable Machine Learning Pipelines Use case

## CREATE A NEW TABLE with specific schema (to show how schema validation fails by default)

In [11]:
spark.sql(
    """CREATE TABLE IF NOT EXISTS glue.test.churn_new
            (State STRING, Account_length INT, Area_code STRING, International_plan STRING, Voicemail_plan STRING, Number_vmail_messages STRING, Total_day_minutes STRING, Total_day_calls STRING, Total_day_charge STRING, Total_eve_minutes STRING, Total_eve_calls STRING, Total_eve_charge STRING, Total_night_minutes STRING, Total_night_calls STRING, Total_night_charge STRING, Total_intl_minutes STRING, Total_intl_calls STRING, Total_intl_charge STRING, Customer_service_calls STRING, Churn STRING) USING iceberg"""
)

DataFrame[]

## Ingestion Job to insert some new data WITHOUT explicit schema checks

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException

def ingest_new_data(new_data_path, iceberg_table_name):
    """
    Ingest new data into an existing Iceberg table. Display an error message if schema doesn't match.
    
    Args:
    - new_data_path: The path to the new dataset to ingest.
    - iceberg_table_name: The name of the existing Iceberg table to ingest data into.
    """
    try:
        # Load the new data
        new_df = spark.read.csv(new_data_path, header=True, inferSchema=True)
        
        # Try to append the new data to the existing table
        new_df.write.format("iceberg").mode("append").saveAsTable(iceberg_table_name)
        print(f"Data from {new_data_path} ingested successfully into {iceberg_table_name}!")
        
    except AnalysisException as ae:
        # Catch any errors during the write operation, including schema mismatch errors
        print(f"Error ingesting data into {iceberg_table_name}: {str(ae)}")
        

# Ingest new data
new_data_path = "churn_etl_new.csv"  # Replace this with your actual path
iceberg_table_name = "glue.test.churn_new"
ingest_new_data(new_data_path, iceberg_table_name)


Error ingesting data into glue.dip.churn_new: Cannot write incompatible data to table 'Iceberg glue.dip.churn_new':
- Cannot safely cast 'Account_length': string to int


## Ingestion Job to insert some new data WITH explicit schema checks

In [12]:
from pyspark.sql.types import StructType, StringType, IntegerType, FloatType, BooleanType, StructField
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException


def validate_and_ingest(new_data_path, iceberg_table_name):
    """
    Validate the schema of the new dataset against the schema of an existing Iceberg table 
    and ingest the new data if the schemas match.
    
    Args:
    - new_data_path: The path to the new dataset to ingest.
    - iceberg_table_name: The name of the existing Iceberg table to validate against.
    """
    try:
        # Load the schema of the existing Iceberg table
        existing_df = spark.table(iceberg_table_name)
        existing_schema = existing_df.schema
        
        # Load the new data
        new_df = spark.read.csv(new_data_path, header=True, inferSchema=True)
        print(new_df.schema)
        
        # Check the schema of the new data against the schema of the existing Iceberg table
        if str(new_df.schema) != str(existing_schema):
            raise ValueError("Schema mismatch detected in the new data!")
        
        # If no errors, append the new data to the existing table
        new_df.write.format("iceberg").mode("append").saveAsTable(iceberg_table_name)
        print(f"Data from {new_data_path} ingested successfully into {iceberg_table_name}!")
        
    except AnalysisException:
        print(f"Error reading the data from {new_data_path} or the Iceberg table {iceberg_table_name}.")
    except ValueError as ve:
        print(ve)

# Validate and ingest new data
new_data_path = "churn_etl.csv"  # Replace this with your actual path
iceberg_table_name = "glue.test.churn"
validate_and_ingest(new_data_path, iceberg_table_name)


StructType([StructField('State', StringType(), True), StructField('Account_length', IntegerType(), True), StructField('Area_code', IntegerType(), True), StructField('International_plan', StringType(), True), StructField('Voicemail_plan', StringType(), True), StructField('Number_vmail_messages', IntegerType(), True), StructField('Total_day_minutes', DoubleType(), True), StructField('Total_day_calls', IntegerType(), True), StructField('Total_day_charge', DoubleType(), True), StructField('Total_eve_minutes', DoubleType(), True), StructField('Total_eve_calls', IntegerType(), True), StructField('Total_eve_charge', DoubleType(), True), StructField('Total_night_minutes', DoubleType(), True), StructField('Total_night_calls', IntegerType(), True), StructField('Total_night_charge', DoubleType(), True), StructField('Total_intl_minutes', DoubleType(), True), StructField('Total_intl_calls', IntegerType(), True), StructField('Total_intl_charge', DoubleType(), True), StructField('Customer_service_cal

## Change Existing Schema: Add 2 new columns

In [29]:
spark.sql("""
    ALTER TABLE glue.test.churn_new 
    ADD COLUMNS (5G_Usage_minutes FLOAT, VoLTE_calls_made INT)
""").show()


23/08/30 00:20:04 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up
++
||
++
++



## READ new data with the new schema

In [ ]:
df_evolved = spark.read.csv("newdatasource.csv")

# Append the new data to the Iceberg table
df_evolved.writeTo("glue.test.churn").append()

## See the New Records

In [20]:
df_evolved.toPandas()

,State,Account_length,Area_code,International_plan,Voicemail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,...,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn,5G_Usage_minutes,VoLTE_calls_made
0,State,NaN,NaN,International_plan,Voicemail_plan,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
1,WI,114.0,415.0,No,Yes,26.0,137.100006,88.0,23.309999,155.699997,...,247.600006,94.0,11.14,11.5,7.0,3.11,2.0,False,NaN,NaN
2,AL,106.0,408.0,No,Yes,29.0,83.599998,131.0,14.210000,203.899994,...,229.500000,73.0,10.33,8.1,3.0,2.19,1.0,False,NaN,NaN
3,VT,60.0,415.0,No,No,0.0,193.899994,118.0,32.959999,85.000000,...,210.100006,134.0,9.45,13.2,8.0,3.56,3.0,False,NaN,NaN
4,WV,159.0,415.0,No,No,0.0,169.800003,114.0,28.870001,197.699997,...,193.699997,82.0,8.72,11.6,4.0,3.13,1.0,False,NaN,NaN
5,CT,184.0,510.0,Yes,No,0.0,213.800003,105.0,36.349998,159.600006,...,139.199997,137.0,6.26,5.0,10.0,1.35,2.0,False,NaN,NaN


# 2. Machine Learning Reproducibility Use case (with Time Travel):

## Find out the table's History

In [21]:
spark.sql("SELECT * FROM glue.test.churn.history").toPandas()

/home/docker/.local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,made_current_at,snapshot_id,parent_id,is_current_ancestor
0,2023-04-20 17:06:12.515,5889239598709613914,NaN,True
1,2023-05-03 19:24:24.418,7869769243560997710,5.889240e+18,True


## Two ways to time travel: via snapshotID or timestamp

In [ ]:
spark.sql("SELECT * FROM glue.test.churn TIMESTAMP AS OF '2023-05-03 19:24:24.418'").toPandas()

In [52]:
spark.sql("SELECT * FROM glue.test.churn VERSION AS OF 5889239598709613914").toPandas()

,State,Account_length,Area_code,International_plan,Voicemail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn
0,LA,117,408,No,No,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,FALSE
1,IN,65,415,No,No,0,129.1,137,21.95,228.5,83,19.42,208.8,111,9.4,12.7,6,3.43,4,TRUE
2,NY,161,415,No,No,0,332.9,67,56.59,317.8,97,27.01,160.6,128,7.23,5.4,9,1.46,4,TRUE
3,SC,111,415,No,No,0,110.4,103,18.77,137.3,102,11.67,189.6,105,8.53,7.7,6,2.08,2,FALSE
4,HI,49,510,No,No,0,119.3,117,20.28,215.1,109,18.28,178.7,90,8.04,11.1,1,3.0,1,FALSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,SC,79,415,No,No,0,134.7,98,22.9,189.7,68,16.12,221.4,128,9.96,11.8,5,3.19,2,FALSE
931,AZ,192,415,No,Yes,36,156.2,77,26.55,215.5,126,18.32,279.1,83,12.56,9.9,6,2.67,2,FALSE
932,WV,68,415,No,No,0,231.1,57,39.29,153.4,55,13.04,191.3,123,8.61,9.6,4,2.59,3,FALSE
933,RI,28,510,No,No,0,180.8,109,30.74,288.8,58,24.55,191.9,91,8.64,14.1,6,3.81,2,FALSE


## Train the ML model on older snapshot

In [39]:
from pyspark.sql import SparkSession
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


# Retrieve a specific snapshot of the Iceberg table from a week ago
snapshot_id_of_interest = 5889239598709613914  

df = spark.read \
    .option("snapshot-id", snapshot_id_of_interest) \
    .format("iceberg") \
    .load("glue.test.churn")

# Convert to Pandas for ML operations
pdf = df.toPandas()

# Split data 
pdf.drop(['State', 'Area_code'], axis=1, inplace=True)

# dummy categorical data
pdf['International_plan']=pdf['International_plan'].replace(['No','Yes'],[0,1])
pdf['Voicemail_plan']=pdf['Voicemail_plan'].replace(['No','Yes'],[0,1])
pdf['Churn']=pdf['Churn'].replace(['FALSE', 'TRUE'],[0,1])

#prepare data 
target = pdf.iloc[: , -1].values
features = pdf.iloc[: , : -1].values

# divide train & test data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20,random_state=101)

# Train a model
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Make predictions and check accuracy
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Model accuracy on data from snapshot {snapshot_id_of_interest}: {accuracy * 100:.2f}%")

Model accuracy on data from snapshot 5889239598709613914: 93.58%


# 3. ML Reproducibility with Tags:

In [45]:
spark.sql(
  """CREATE OR REPLACE TEMPORARY VIEW new_train_data USING csv
            OPTIONS (path "newtrainingdata.csv", header true)"""
)

spark.sql(
  "INSERT INTO glue.test.churn_new SELECT * FROM new_train_data"
)

DataFrame[]

## CREATE A TAG

In [49]:
spark.sql("ALTER TABLE glue.test.churn_new CREATE TAG June_23")

23/08/30 01:57:14 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up


DataFrame[]

## Query all the tags/branches

In [50]:
spark.sql("SELECT * FROM glue.test.churn_new.refs").toPandas()

,name,type,snapshot_id,max_reference_age_in_ms,min_snapshots_to_keep,max_snapshot_age_in_ms
0,main,BRANCH,2496871934354606665,NaN,NaN,NaN
1,June_23,TAG,2496871934354606665,NaN,NaN,NaN


## Refer to this specific tag (version of data) for model training

In [57]:
from pyspark.sql import SparkSession
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


df_new = spark.read \
    .option("tags", 'June_23') \
    .format("iceberg") \
    .load("glue.test.churn_new")

pdf = df_new.toPandas()

# Split data 
pdf.drop(['State', 'Area_code'], axis=1, inplace=True)

# dummy categorical data
pdf['International_plan']=pdf['International_plan'].replace(['No','Yes'],[0,1])
pdf['Voicemail_plan']=pdf['Voicemail_plan'].replace(['No','Yes'],[0,1])
pdf['Churn']=pdf['Churn'].replace(['FALSE', 'TRUE'],[0,1])

#prepare data 
target = pdf.iloc[: , -1].values
features = pdf.iloc[: , : -1].values

# divide train & test data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20,random_state=101)

# Train a model
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Make predictions and check accuracy
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Model accuracy on data from snapshot {snapshot_id_of_interest}: {accuracy * 100:.2f}%")

# 4. ML Experimentation Use case

## Create a new BRANCH

In [59]:
spark.sql("ALTER TABLE glue.test.churn_new CREATE BRANCH Churn_PremiumExp")

23/08/30 17:08:36 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up


DataFrame[]

## Write new data to this specific Branch

In [61]:
churn_schema = spark.table("glue.test.churn_new").schema

 # Load the new data
df_exp = spark.read.csv("churndata.csv", header=True, inferSchema=True)

df_exp.write.format("iceberg").mode("append").save("glue.test.churn_new.branch_Churn_PremiumExp")

## Query the main dataset

In [63]:
spark.table("glue.test.churn_new").toPandas()

,State,Account_length,Area_code,International_plan,Voicemail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn
0,LA,117,408,No,No,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,FALSE
1,IN,65,415,No,No,0,129.1,137,21.95,228.5,83,19.42,208.8,111,9.4,12.7,6,3.43,4,TRUE
2,NY,161,415,No,No,0,332.9,67,56.59,317.8,97,27.01,160.6,128,7.23,5.4,9,1.46,4,TRUE
3,SC,111,415,No,No,0,110.4,103,18.77,137.3,102,11.67,189.6,105,8.53,7.7,6,2.08,2,FALSE
4,HI,49,510,No,No,0,119.3,117,20.28,215.1,109,18.28,178.7,90,8.04,11.1,1,3.0,1,FALSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235,OH,87,510,No,No,0,153.3,106,26.06,224.5,117,19.08,273.4,152,12.3,8.9,5,2.4,2,FALSE
1236,KS,73,408,No,No,0,194.8,112,33.12,167.2,85,14.21,100.3,61,4.51,10.8,6,2.92,1,FALSE
1237,IN,7,415,No,No,0,206.7,87,35.14,281.1,83,23.89,158.5,77,7.13,11.0,5,2.97,3,FALSE
1238,OR,89,415,No,Yes,12,188.0,105,31.96,151.3,107,12.86,201.9,132,9.09,10.5,3,2.84,2,FALSE


## Query the experimental branch

In [64]:
spark.read.format("iceberg").load("glue.test.churn_new.branch_Churn_PremiumExp").toPandas()

,State,Account_length,Area_code,International_plan,Voicemail_plan,Number_vmail_messages,Total_day_minutes,Total_day_calls,Total_day_charge,Total_eve_minutes,Total_eve_calls,Total_eve_charge,Total_night_minutes,Total_night_calls,Total_night_charge,Total_intl_minutes,Total_intl_calls,Total_intl_charge,Customer_service_calls,Churn
0,LA,117,408,No,No,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,false
1,IN,65,415,No,No,0,129.1,137,21.95,228.5,83,19.42,208.8,111,9.4,12.7,6,3.43,4,true
2,NY,161,415,No,No,0,332.9,67,56.59,317.8,97,27.01,160.6,128,7.23,5.4,9,1.46,4,true
3,SC,111,415,No,No,0,110.4,103,18.77,137.3,102,11.67,189.6,105,8.53,7.7,6,2.08,2,false
4,HI,49,510,No,No,0,119.3,117,20.28,215.1,109,18.28,178.7,90,8.04,11.1,1,3.0,1,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2170,WI,114,415,No,Yes,26,137.1,88,23.31,155.7,125,13.23,247.6,94,11.14,11.5,7,3.11,2,FALSE
2171,AL,106,408,No,Yes,29,83.6,131,14.21,203.9,131,17.33,229.5,73,10.33,8.1,3,2.19,1,FALSE
2172,VT,60,415,No,No,0,193.9,118,32.96,85.0,110,7.23,210.1,134,9.45,13.2,8,3.56,3,FALSE
2173,WV,159,415,No,No,0,169.8,114,28.87,197.7,105,16.8,193.7,82,8.72,11.6,4,3.13,1,FALSE


In [ ]:
from pyspark.sql import SparkSession
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


df_prem = spark.read \
    .option("branch", 'Churn_PremiumExp') \
    .format("iceberg") \
    .load("glue.test.churn_new")

# Convert to Pandas for ML operations
pdf = df_prem.toPandas()

# Split data 
pdf.drop(['State', 'Area_code'], axis=1, inplace=True)

# dummy categorical data
pdf['International_plan']=pdf['International_plan'].replace(['No','Yes'],[0,1])
pdf['Voicemail_plan']=pdf['Voicemail_plan'].replace(['No','Yes'],[0,1])
pdf['Churn']=pdf['Churn'].replace(['FALSE', 'TRUE'],[0,1])

#prepare data 
target = pdf.iloc[: , -1].values
features = pdf.iloc[: , : -1].values

# divide train & test data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20,random_state=101)

# Train a model
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Make predictions and check accuracy
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Model accuracy on data from snapshot {snapshot_id_of_interest}: {accuracy * 100:.2f}%")

In [81]:
# New:
spark.sql("ALTER TABLE glue.test.churn DROP BRANCH Churn_PremiumExp")

23/03/27 17:40:54 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up


DataFrame[]